<a href="https://colab.research.google.com/github/annechris13/Master-Thesis/blob/master/performance_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import pandas as pd
from sklearn.datasets import make_spd_matrix
# import random
import warnings
warnings.filterwarnings('error')
import time
import cvxpy as cp
import random as r
from scipy.linalg import sqrtm

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!pip install import-ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=6d602abd0da05f341dee42f616a067189a2d3a1c59c3f2e6cf69820b0d7576fa
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


Exception ignored in: <_io.FileIO name='/usr/local/lib/python3.6/dist-packages/import_ipynb-0.1.3.dist-info/top_level.txt' mode='rb' closefd=True>


In [5]:
%cd "/content/gdrive/My Drive/Colab Notebooks"
import mpc_class
import qpth_class

/content/gdrive/My Drive/Colab Notebooks
importing Jupyter notebook from mpc_class.ipynb
importing Jupyter notebook from qpth_class.ipynb


In [6]:
%cd "/content"
df=pd.read_csv('gdrive/My Drive/test_cases.csv')
seeds=df["seed"].astype(int)
# df.head()

/content


In [0]:
def qp_cvxpy(n,P,q,G,h,A,b):
    x= cp.Variable(n)
    objective=cp.Minimize((1/2)*cp.sum_squares(P*x) + q.T @ x)
    constraints=[G@x <=h,A@x ==b]
    prob=cp.Problem(objective,constraints)
    prob.solve(verbose=False)
    cvxpy_optimum= prob.value
    cvxpy_solution=x.value
    return cvxpy_optimum,cvxpy_solution

# Test on individual problems

In [0]:
n=6
m=6
p=3
nbatch=1

In [9]:
cvxpy_time=0.0
mpc_time=0.0
qpth_time=0.0
columns=["seed"]+[str(i)+"cp" for i in range(n)]
results=pd.DataFrame([],columns=columns)
for k,s in enumerate(seeds):
    r.seed(s)
    P=make_spd_matrix(n, random_state=s)
    q=np.array([r.randint(0,9) for i in range(n)]).reshape(n).astype(float)
    G=np.array([r.randint(0,9)*((-1)**r.randint(0,1)) for i in range(m*n)]).reshape(m,n).astype(float)
    h=np.array([0 for i in range(m)]).reshape(n).astype(float)
    A=np.array([r.randint(0,9) for i in range(p*n)]).reshape(p,n).astype(float)
    b=np.array([r.randint(0,9) for i in range(p)]).reshape(p).astype(float)
    
    P_tch=torch.from_numpy(P)
    q_tch=torch.from_numpy(q)
    G_tch=torch.from_numpy(G)
    h_tch=torch.from_numpy(h)
    A_tch=torch.from_numpy(A)
    b_tch=torch.from_numpy(b)
    
    #get cvxpy solution
    P = sqrtm(P)
    start=time.time()
    x,y=qp_cvxpy(n,P,q,G,h,A,b)
    cvxpy_time+=time.time()-start
    re=pd.Series({"seed":s})
    for i in range(n):
        re=re.append(pd.Series({str(i)+"cp":np.round(y[i],6)}))
    
    #get mpc solution
    try:
      start=time.time()
      solution = mpc_class.opt(P_tch.view(nbatch,n,n), q_tch.view(nbatch,n), G_tch.view(nbatch,m,n), h_tch.view(nbatch,m), A_tch.view(nbatch,p,n), b_tch.view(nbatch,p))
      mpc_time+=time.time()-start
      # print(solution.numpy().reshape(-1))
      qpth_solution=solution.numpy().reshape(-1)
    except:
        qpth_solution=[0]*n
    for i in range(n):
        re=re.append(pd.Series({str(i)+"mpc":np.round(qpth_solution[i],6)}))

    #get qpth solution
    # try:
    start=time.time()
    solution,_,_,_ = qpth_class.qpth_opt(P_tch.view(nbatch,n,n), q_tch.view(nbatch,n), G_tch.view(nbatch,m,n), h_tch.view(nbatch,m), A_tch.view(nbatch,p,n), b_tch.view(nbatch,p))
    qpth_time+=time.time()-start
      # print(solution.numpy().reshape(-1))
    qpth_solution=solution.detach().numpy()[0]
    # except:
        # qpth_solution=[0]*n
    for i in range(n):
        re=re.append(pd.Series({str(i)+"qpt":np.round(qpth_solution[i],6)}))

    results=results.append(re,ignore_index=True )

qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate 

In [10]:
print("cvxpy time: ", cvxpy_time)
print("mpc time: ", mpc_time)
print("qpth time: ", qpth_time)

cvxpy time:  6.490458250045776
mpc time:  16.516886472702026
qpth time:  17.47646713256836


In [11]:
error_qpt=np.zeros(len(results))
error_mpc=np.zeros(len(results))
for i in range(n):
    error_qpt+=results[str(i)+"cp"]-results[str(i)+"qpt"]
    error_mpc+=results[str(i)+"cp"]-results[str(i)+"mpc"]
results["cp-qpt-error"]=error_qpt   
results["cp-qpt-er_flag"]=error_qpt!=0

results["cp-mpc-error"]=error_mpc    
results["cp-mpc-er_flag"]=error_mpc!=0

print("No of solved problems",len(seeds))
print("No of inaccurate results from qpth",len(error_qpt[error_qpt!=0]))
print("No of inaccurate results from mpc",len(error_mpc[error_mpc!=0]))

No of solved problems 639
No of inaccurate results from qpth 22
No of inaccurate results from mpc 1


In [12]:
results[results["cp-mpc-er_flag"]]

,seed,0cp,1cp,2cp,3cp,4cp,5cp,0mpc,0qpt,1mpc,1qpt,2mpc,2qpt,3mpc,3qpt,4mpc,4qpt,5mpc,5qpt,cp-qpt-error,cp-qpt-er_flag,cp-mpc-error,cp-mpc-er_flag
318,5130.0,-20.239846,19.914573,-5.128942,22.540444,-9.099782,5.446464,-20.141635,-0.336779,19.815589,-0.241715,-5.098859,1.080021,22.431559,0.297595,-9.057034,-0.320653,5.423004,0.776832,12.17761,True,0.060287,True


In [13]:
results[results["cp-qpt-er_flag"]]

,seed,0cp,1cp,2cp,3cp,4cp,5cp,0mpc,0qpt,1mpc,1qpt,2mpc,2qpt,3mpc,3qpt,4mpc,4qpt,5mpc,5qpt,cp-qpt-error,cp-qpt-er_flag,cp-mpc-error,cp-mpc-er_flag
15,360.0,-0.400090,-8.847799,4.028417,-5.439977,1.761969,2.236549,-0.400090,0.464182,-8.847799,-1.552062,4.028417,1.138961,-5.439977,-0.798860,1.761969,0.279653,2.236549,0.074129,-6.266934,True,0.000000,False
18,400.0,5.238220,-1.923429,5.077225,5.727094,-6.282723,6.267016,5.238220,1.262468,-1.923429,1.062144,5.077225,-0.506128,5.727094,2.006004,-6.282723,-1.288326,6.267016,0.155147,11.412094,True,0.000000,False
72,1340.0,-1.221996,3.452352,-1.260098,-4.570263,-1.121232,2.466149,-1.221996,0.193232,3.452352,0.979446,-1.260098,-0.436470,-4.570263,-1.311860,-1.121232,-0.307930,2.466149,0.882552,-2.254058,True,0.000000,False
174,2920.0,-9.338583,-9.881890,5.220472,5.653543,6.244094,10.157480,-9.338583,-7.439791,-9.881890,-7.530245,5.220472,4.875014,5.653543,3.822770,6.244094,4.354812,10.157480,8.188102,1.784454,True,0.000000,False
183,3070.0,-4.483220,2.040414,7.561966,8.055154,-8.131626,-3.116473,-4.483220,0.555620,2.040414,-0.247828,7.561966,0.545141,8.055154,0.664442,-8.131626,-0.273798,-3.116473,-0.101710,0.784348,True,0.000000,False
190,3150.0,-18.274590,-27.152459,8.861475,9.636066,7.383607,7.514754,-18.274590,0.315030,-27.152459,-0.230616,8.861475,0.209725,9.636066,0.116471,7.383607,0.004432,7.514754,0.509967,-12.956156,True,0.000000,False
203,3330.0,-17.249111,-6.010441,7.250754,-11.677336,1.789343,17.291923,-17.249111,-1.343228,-6.010441,0.638510,7.250754,-0.302427,-11.677336,0.013072,1.789343,0.574842,17.291923,0.100492,-8.286129,True,0.000000,False
221,3650.0,-2.358418,-2.869544,-4.816039,-1.072076,7.109924,-3.063688,-2.358418,-2.352926,-2.869544,-2.883133,-4.816039,-4.815594,-1.072076,-1.070506,7.109924,7.103760,-3.063688,-3.057295,0.005853,True,0.000000,False
285,4590.0,3.362302,-1.489500,-2.054629,-1.207035,2.713798,-0.921727,3.362302,0.488899,-1.489500,0.856958,-2.054629,-0.518210,-1.207035,-0.239803,2.713798,0.053463,-0.921727,0.208193,-0.446291,True,0.000000,False
318,5130.0,-20.239846,19.914573,-5.128942,22.540444,-9.099782,5.446464,-20.141635,-0.336779,19.815589,-0.241715,-5.098859,1.080021,22.431559,0.297595,-9.057034,-0.320653,5.423004,0.776832,12.177610,True,0.060287,True


# Test on Batch of problems

In [0]:
import random

In [0]:
ip="csv"
df=df.iloc[:,:8]
nbatch=len(df)
nx=6
nineq=6
neq=3
Q=[]
p=[]
G=[]
h=[]
A=[]
b=[]
for i in range(nbatch):
  seed=int(df["seed"][i])
  random.seed(seed)
  Q.append(make_spd_matrix(nx,random_state=seed).reshape(1,-1))
  p.append([random.randint(0,9) for i in range(nx)])#.astype(float))
  G.append([random.randint(0,9)*((-1)**random.randint(0,1)) for i in range(nineq*nx)])#.astype(float))
  h.append([0 for i in range(nineq)])#.astype(float))
  A.append([random.randint(0,9) for i in range(neq*nx)])#.astype(float))
  b.append([random.randint(0,9) for i in range(neq)])#.astype(float)  )

Q=torch.tensor(Q).view(nbatch,nx,nx).type(torch.DoubleTensor)
p=torch.tensor(p).view(nbatch,nx).type(torch.DoubleTensor)
G=torch.tensor(G).view(nbatch,nineq,nx).type(torch.DoubleTensor)
h=torch.tensor(h).view(nbatch,nineq).type(torch.DoubleTensor)
A=torch.tensor(A).view(nbatch,neq,nx).type(torch.DoubleTensor)
b=torch.tensor(b).view(nbatch,neq).type(torch.DoubleTensor)


In [0]:
# import cProfile, pstats, StringIO
# pr = cProfile.Profile()


In [17]:
start=time.time()
x_mpc=mpc_class.opt(Q,p,G,h,A,b)
mpc_time=time.time()-start
start=time.time()
x_qpt,_,_,_=qpth_class.qpth_opt(Q,p,G,h,A,b)
qpth_time=time.time()-start
print("mpc time: ", mpc_time)
print("qpth time: ", qpth_time)

mpc time:  2.713951587677002
qpth time:  2.456608772277832


In [18]:
cols_mpc=[str(i)+"mpc" for i in range(nx)]
cols_qpt=[str(i)+"qpt" for i in range(nx)]
error_qpt=np.zeros(nbatch)
error_mpc=np.zeros(nbatch)
new_mpc=pd.DataFrame(x_mpc.numpy().round(6).reshape(nbatch,-1),columns=cols_mpc)
new_qpt=pd.DataFrame(x_qpt.numpy().round(6).reshape(nbatch,-1),columns=cols_qpt)
tmp=df.copy()
for i in range(nx):
    col_mpc=cols_mpc[i]
    col_qpt=cols_qpt[i]
    tmp[col_mpc]=new_mpc[col_mpc].copy()
    tmp[col_qpt]=new_qpt[col_qpt].copy()
    error_mpc+=np.round(tmp[str(i)+"cp"],6)-np.round(tmp[col_mpc],6)
    error_qpt+=np.round(tmp[str(i)+"cp"],6)-np.round(tmp[col_qpt],6)
tmp["cp-mpc-error"]=error_mpc
tmp["cp-mpc-er_flag"]=error_mpc!=0
tmp["cp-qpt-error"]=error_qpt
tmp["cp-qpt-er_flag"]=error_qpt!=0
print("No.of problems solved: ",len(tmp))
print("No.of errors from MPC Method: ",len(error_mpc[error_mpc!=0])," (",round(len(error_mpc[error_mpc!=0])*100/len(tmp),2),")")
print("No.of errors from qpth: ",len(error_qpt[error_qpt!=0])," (",round(len(error_qpt[error_qpt!=0])*100/len(tmp),2),")")

No.of problems solved:  639
No.of errors from MPC Method:  1  ( 0.16 )
No.of errors from qpth:  2  ( 0.31 )


In [19]:
tmp[tmp["cp-qpt-er_flag"]]

,Unnamed: 0,seed,0cp,1cp,2cp,3cp,4cp,5cp,0mpc,0qpt,1mpc,1qpt,2mpc,2qpt,3mpc,3qpt,4mpc,4qpt,5mpc,5qpt,cp-mpc-error,cp-mpc-er_flag,cp-qpt-error,cp-qpt-er_flag
318,318,5130.0,-20.239846,19.914573,-5.128942,22.540444,-9.099782,5.446464,-20.141635,-20.141635,19.815589,19.815589,-5.098859,-5.098859,22.431559,22.431559,-9.057034,-9.057034,5.423004,5.423004,0.060287,True,0.060287,True
375,375,5910.0,0.034153,0.340292,0.233124,0.122422,-0.695894,1.752989,0.034153,0.050554,0.340292,0.325828,0.233124,0.265256,0.122422,0.132445,-0.695894,-0.709966,1.752989,1.756230,0.000000,False,-0.033261,True
